In [ ]:
# Useful Links
# 1. https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
# 2. https://machinelearningknowledge.ai/keras-implementation-of-resnet-50-architecture-from-scratch/
# 3. https://towardsdatascience.com/resnets-for-cifar-10-e63e900524e0 (Understanding the resnet architecture for cifar10)


In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Load necessary libraries
for i in range(5):
  import os
  import pandas as pd
  import numpy as np
  #import cv2
  import tensorflow as tf
  from tensorflow import keras
  from tensorflow.keras import backend as K
  from tensorflow.keras.optimizers import SGD, Adam
  #from google.colab.patches import cv2_imshow
  from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint, ReduceLROnPlateau, CSVLogger
  from tensorflow.keras.models import Sequential, Model,load_model
  from tensorflow.keras.preprocessing.image import ImageDataGenerator
  from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D,Dropout
  from tensorflow.keras.preprocessing import image
  from tensorflow.keras.initializers import glorot_uniform
  from matplotlib import pyplot as plt
  from tensorflow.keras.utils import to_categorical, plot_model



  def load_dataset(dName="CIFAR10"):
      dataset = None
      num_classes = None
      if dName == "CIFAR10":
          num_classes = 10
          dataset = tf.keras.datasets.cifar10.load_data()
      if dName == "CIFAR100":
          num_classes = 100
          dataset = tf.keras.datasets.cifar100.load_data()
      (X_train, y_train), (X_test, y_test) = dataset
      # Convert target value to categorical values
      # One-hot-encoded target values
      y_train = to_categorical(y_train,num_classes)
      y_test = to_categorical(y_test, num_classes)
      
      return (X_train, y_train),(X_test, y_test)
  def divideDataset(X_train, y_train, X_test,y_test):
      dataLength = X_train.shape[0]
      trainLen=0
      dataTrain = []
      dataTest = []
      percent = 0.2
      while(trainLen<dataLength):
          trainLen = int(X_train.shape[0]*percent)
          #print(X_train[:trainLen].shape)
          train = (X_train[:trainLen],y_train[:trainLen])
          retriveLen = int(X_test.shape[0]*percent)
          test = (X_test[:retriveLen],y_test[:retriveLen])
          #print(tuple(train))
          dataTrain.append(train)
          dataTest.append(test)
          #print("\n")
          percent +=0.2
      return dataTrain, dataTest
  (X_train, y_train),(X_test, y_test) = load_dataset()
  dataTrain, dataTest = divideDataset(X_train, y_train, X_test, y_test) # this contains the list of 5 different datasets
  def normalizeInput(X_train,X_test):
      X_train = X_train.astype('float32')
      X_test = X_test.astype('float32')
      mean = np.mean(X_train, axis=(0,1,2,3))
      std = np.std(X_train,axis=(0,1,2,3))
      X_train = (X_train-mean)/(std+1e-7)
      X_test = (X_test-mean)/(std+1e-7)
      
      return X_train, X_test
  ##### Include Little Data Augmentation 
  '''
  batch_size = 64 # try several values

  train_DataGen = tf.keras.preprocessing.image.ImageDataGenerator(zoom_range=0.2, 
                                                                  width_shift_range=0.1, 
                                                                  height_shift_range = 0.1, 
                                                                  horizontal_flip=True)
  
  valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

  train_set_conv = train_DataGen.flow(X_train, y_train, batch_size=batch_size) # train_lab is categorical 
  valid_set_conv = valid_datagen.flow(X_test, y_test, batch_size=batch_size) # so as valid_lab
  '''
  X_train, X_test = normalizeInput(dataTrain[0][0], dataTest[0][0])
  y_train = dataTrain[0][1]
  y_test = dataTest[0][1]
  # ResNet Implementation
  # Code Link: https://github.com/Gurupradeep/CIFAR-10-Object-Recognition-in-Images/blob/master/Models/ResNet.ipynb
  #Defining Callback functions which will be called by model during runtime when specified condition satisfies

  #lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=2, min_lr=0.5e-6)
  #csv_logger = CSVLogger('ResNet50_without_dropout_without_conv_without_pool0.csv')
  early_stopper = EarlyStopping(min_delta=0.001, patience=30)
  model_chekpoint = ModelCheckpoint("ResNet50_without_dropout_without_conv_without_pool.hdf5",monitor = 'val_loss',verbose = 1,save_best_only=True)
  #model Parameters
  batch_size = 64
  data_augmentation = False
  epochs = 30
  '''
  if data_augmentation :
      print("-------------Using Data augmentation------------")
      # This will do preprocessing and realtime data augmentation:
      datagen = ImageDataGenerator(
          featurewise_center=False,  # set input mean to 0 over the dataset
          samplewise_center=False,  # set each sample mean to 0
          featurewise_std_normalization=False,  # divide inputs by std of the dataset
          samplewise_std_normalization=False,  # divide each input by its std
          zca_whitening=False,  # apply ZCA whitening
          rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
          width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
          height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
          horizontal_flip=True,  # randomly flip images
          vertical_flip=False)  # randomly flip images
      
      datagen.fit(X_train)
      model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                          steps_per_epoch=X_train.shape[0] // batch_size,
                          epochs=epochs,verbose=1,validation_data=(X_test,y_test),callbacks = [lr_reducer,csv_logger])
      
  else :
      print("-----Not Using Data augmentation---------------")
      model.fit(X_train, y_train,
                batch_size=batch_size,
                epochs=epochs,
                validation_data=(X_test, y_test),
                shuffle=True,callbacks = [csv_logger])
                '''
  # RESNET50 WITH SKIP CONNECTION 2: Implemented by Lokesh
  def createStageOne(x, filters,kernels, strides,paddings,counter):
    '''
    Argument
    x - input
    f - filter_size
    k - kernel size
    s - stride
    p - padding
    '''
    f1, f2 = filters
    k1, k2 = kernels
    s1, s2 = strides
    p1, p2 = paddings
    block_name = 'stage_1'+counter
    x_skip = x
    x = Conv2D(f1, kernel_size=(k1,k1),strides=(s1,s1),padding=p1, name=block_name+'a')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(f2,kernel_size=(k2,k2),strides=(s2,s2),padding=p2,name=block_name+'b')(x)
    x = BatchNormalization()(x)

    x_skip = Conv2D(f2,kernel_size=(1,1),strides=(1,1),padding='valid')(x_skip)
    x_skip = BatchNormalization()(x_skip)
    x = Add()([x,x_skip])
    x = Activation('relu')(x)
    #print("Stage 1:", x.shape)

    return x

  def createStageTwo(x,filters,kernels,strides,paddings,counter):
    '''
    Arguments
    x - input
    filters - filter size
    kernels - kernel size
    strides - stride size
    paddings - padding size
    '''
    #print("stage 2", x.shape)
    f1, f2 = filters
    k1, k2 = kernels
    s1, s2 = strides
    p1, p2 = paddings
    block_name = 'stage_2'+counter
    x_skip = x
    x = Conv2D(f1,kernel_size=(k1,k1),strides=(s1,s1),padding=p1,name=block_name+'a')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(f2,kernel_size=(k2,k2),strides=(s2,s2),padding=p2,name=block_name+'b')(x)
    x = BatchNormalization()(x)
    if x.shape[1] != x_skip.shape[1]:
      x_skip = Conv2D(f2,kernel_size=(1,1),strides=(2,2),padding='valid')(x_skip)
    else:
      x_skip = Conv2D(f2,kernel_size=(1,1),strides=(1,1),padding='valid')(x_skip)
    x_skip = BatchNormalization()(x_skip)
    x = Add()([x,x_skip])
    x = Activation('relu')(x)
    #print('Stage 2:',x.shape)
    return x
  def createStageThree(x,filters,kernels,strides,paddings,counter):
    '''
    Arguments
    x - input
    filters - filter size
    kernels - kernel size
    strides - stride size
    paddings - padding size
    '''

    f1, f2 = filters
    k1, k2 = kernels
    s1, s2 = strides
    p1, p2 = paddings
    block_name = 'stage_3'+counter
    x_skip = x
    x = Conv2D(f1,kernel_size=(k1,k1),strides=(s1,s1),padding=p1,name=block_name+'a')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(f2,kernel_size=(k2,k2),strides=(s2,s2),padding=p2,name=block_name+'b')(x)
    x = BatchNormalization()(x)
    if x.shape[1] != x_skip.shape[1]:
      x_skip = Conv2D(f2,kernel_size=(1,1),strides=(2,2),padding='valid')(x_skip)
    else:
      x_skip = Conv2D(f2,kernel_size=(1,1),strides=(1,1),padding='valid')(x_skip)
    x_skip = BatchNormalization()(x_skip)
    x = Add()([x,x_skip])
    x = Activation('relu')(x)
    #print('Stage 3:',x.shape)
    return x
  def createStageFour(x,filters,kernels,strides,paddings,counter):
    '''
    Arguments
    x - input
    filters - filter size
    kernels - kernel size
    strides - stride size
    paddings - padding size
    '''
    f1, f2 = filters
    k1, k2 = kernels
    s1, s2 = strides
    p1, p2 = paddings
    block_name = 'stage_4'+counter
    x_skip = x
    x = Conv2D(f1,kernel_size=(k1,k1),strides=(s1,s1),padding=p1,name=block_name+'a')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(f2,kernel_size=(k2,k2),strides=(s2,s2),padding=p2,name=block_name+'b')(x)
    x = BatchNormalization()(x)
    if x.shape[1] != x_skip.shape[1]:
      x_skip = Conv2D(f2,kernel_size=(1,1),strides=(2,2),padding='valid')(x_skip)
    else:
      x_skip = Conv2D(f2,kernel_size=(1,1),strides=(1,1),padding='valid')(x_skip)
    x_skip = BatchNormalization()(x_skip)
    x = Add()([x,x_skip])
    x = Activation('relu')(x)

    return x
  def resnet50():
    inputs = Input(shape = (32,32,3), name = "image_input")
    x = ZeroPadding2D((3,3))(inputs)
    # Initial Convolutional block
    
    x = Conv2D(64, kernel_size=(7,7), strides = (2,2))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3,3),strides=(2,2))(x)
    x = createStageOne(x,[64,64],[1,3],[1,1],['valid','same'],'1')
    x = createStageOne(x,[256,64],[1,1],[1,1],['valid','valid'],'2')
    x = createStageOne(x,[64,256],[3,1],[1,1],['same','valid'],'3')
    x = createStageOne(x,[64,64],[1,3],[1,1],['valid','same'],'4')
    x = createStageTwo(x,[256,128],[1,1],[1,2],['valid','valid'],'5')
    x = createStageTwo(x,[128,512],[3,1],[1,1],['same','valid'],'6')
    x = createStageTwo(x,[128,128],[1,3],[1,1],['valid','same'],'7')
    x = createStageTwo(x,[512,128],[1,1],[1,1],['valid','valid'],'8')
    x = createStageTwo(x,[128,512],[3,1],[1,1],['same','valid'],'9')
    x = createStageTwo(x,[128,128],[1,3],[1,1],['valid','same'],'10')
    x = createStageThree(x,[512,256],[1,1],[1,2],['valid','valid'],'11')
    x = createStageThree(x,[256,1024],[3,1],[1,1],['same','valid'],'12')
    x = createStageThree(x,[256,256],[1,3],[1,1],['valid','same'],'13')
    x = createStageThree(x,[1024,256],[1,1],[1,1],['valid','valid'],'14')
    x = createStageThree(x,[256,1024],[3,1],[1,1],['same','valid'],'15')
    x = createStageThree(x,[256,256],[1,3],[1,1],['valid','same'],'16')
    x = createStageThree(x,[1024,256],[1,1],[1,1],['valid','valid'],'17')
    x = createStageThree(x,[256,1024],[3,1],[1,1],['same','valid'],'18')
    x = createStageThree(x,[256,256],[1,3],[1,1],['valid','same'],'19')
    x = createStageFour(x,[1024,512],[1,1],[1,2],['valid','valid'],'20')
    x = createStageFour(x,[512,2048],[3,1],[1,1],['same','valid'],'21')
    x = createStageFour(x,[512,512],[1,3],[1,1],['valid','same'],'22')
    x = createStageFour(x,[2048,512],[1,1],[1,1],['valid','valid'],'23')
    x = createStageFour(x,[512,2048],[3,1],[1,1],['same','valid'],'24')
    x = AveragePooling2D((2,2),name='avg_pool',padding='same')(x)
    x = Flatten()(x)
    x = Dense(10,activation='softmax')(x)
    model = Model(inputs=inputs,outputs=x, name='LResNet50')

    return model

  model = resnet50()
  model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
  #model.summary()
  X_train, X_test = normalizeInput(dataTrain[i][0], dataTest[i][0])
  y_train = dataTrain[i][1]
  y_test = dataTest[i][1]
  print("Len: ", len(X_train))
  earlyStop = EarlyStopping(monitor='loss',patience=5,mode='auto')
  csvLogger = CSVLogger('/content/drive/MyDrive/PhD Studies/fall21/ADVML/results/skip2/'+str(i)+'.csv')
  history = model.fit(X_train,y_train, epochs=300,batch_size=1024,validation_split=0.3,verbose=1, callbacks=[csvLogger,earlyStop])

Len:  10000
Epoch 1/300
7/7 [==============================] - 16s 850ms/step - loss: 3.2429 - accuracy: 0.1106 - val_loss: 2.3045 - val_accuracy: 0.0933
Epoch 2/300
7/7 [==============================] - 2s 246ms/step - loss: 2.4724 - accuracy: 0.1453 - val_loss: 2.3055 - val_accuracy: 0.0933
Epoch 3/300
7/7 [==============================] - 2s 247ms/step - loss: 2.2261 - accuracy: 0.1766 - val_loss: 2.3106 - val_accuracy: 0.0933
Epoch 4/300
7/7 [==============================] - 2s 246ms/step - loss: 2.1304 - accuracy: 0.2046 - val_loss: 2.3211 - val_accuracy: 0.0933
Epoch 5/300
7/7 [==============================] - 2s 247ms/step - loss: 2.0447 - accuracy: 0.2249 - val_loss: 2.3449 - val_accuracy: 0.0933
Epoch 6/300
7/7 [==============================] - 2s 246ms/step - loss: 1.9519 - accuracy: 0.2594 - val_loss: 2.3745 - val_accuracy: 0.0933
Epoch 7/300
7/7 [==============================] - 2s 246ms/step - loss: 1.8425 - accuracy: 0.2896 - val_loss: 2.3915 - val_accuracy: 0.0997
